# Compute optimal individual parameters for GMAC metric

In [ ]:
from zurich_move_data_extraction import *
from activity_count_function import *
from utilities import *
from gm_function import *
from individual_analysis_ac_functions import *
from extract_mask_from_video import *
from individual_analysis_fs_functions import *
from group_analysis_functions import *

In [ ]:
#CHANGE VALUES HERE TODO
# H for Healthy and S for Stroke 
participant_id = 'H005'
participant_group = 'H'
using_Axivity = False

In [ ]:
#PATH# 
initial_path = '../CreateStudy'
participant_path = os.path.join(initial_path, participant_id)
print(participant_path)

In [ ]:
# Extract dataset from the participant JSON file 
participant_data = load_participant_json(participant_id, initial_path)
# Get the correct handiness 
# For stroke, dominant hand = non affected hand 
dominant_hand = get_dominant_hand(participant_data)

In [ ]:
# Get the data from IMU csv files 
ndh_data, chest_data, dh_data, GT_mask_25Hz_NDH, GT_mask_25Hz_DH = get_data(participant_path, dominant_hand)
# Ensure the array sizes consistency between masks and IMU data
print (len(GT_mask_25Hz_NDH)*2, len(ndh_data))

In [ ]:
# Get useful variables
sampling_freq = 50
acc_ndh = ndh_data[['acc_x', 'acc_y', 'acc_z']]
acc_dh = dh_data[['acc_x', 'acc_y', 'acc_z']]
gyro_ndh = ndh_data[['gyro_x', 'gyro_y', 'gyro_z']]
gyro_dh = dh_data[['gyro_x', 'gyro_y', 'gyro_z']]
if not using_Axivity:
    mag_ndh = ndh_data[['magneto_x', 'magneto_y', 'magneto_z']]
    quat_sensor_ndh = ndh_data[['quat_0', 'quat_1', 'quat_2','quat_3']]
    quat_sensor_dh = dh_data[['quat_0', 'quat_1', 'quat_2','quat_3']]
frequency_GMAC = 1 # Hz
FPS_mask = 25 # Hz
frequency_euler_angles = 50 # Hz 

## compute GMAC Counts

In [ ]:
#Compute Actigraph Activity Count using Brond calculation TODO
# raw code here: https://github.com/jbrond/ActigraphCounts/blob/master/R/Python_G_to_sec.py
count_brond_ndh, processed_data_ndh = get_counts_brond(ndh_data)
count_brond_dh, processed_data_dh = get_counts_brond(dh_data)
GMAC_ndh = np.array(count_brond_ndh['AC Brond'])
GMAC_dh = np.array(count_brond_dh['AC Brond'])

### GT MASK Resampling

In [ ]:
# Replace all WBM (whole-body movements) by NF movements inside the GT mask @ 25 Hz 
# Could be changed in the future (e.g. remove all WBM movements)
GT_mask_NDH_25Hz = replace_wbm_with_nf(GT_mask_25Hz_NDH).ravel()
GT_mask_DH_25Hz = replace_wbm_with_nf(GT_mask_25Hz_DH).ravel()

In [ ]:
# Downsample GT masks to be used later on for GMAC (1Hz) computation
GT_mask_NDH_1Hz = resample_mask(GT_mask_NDH_25Hz, FPS_mask, frequency_GMAC)
GT_mask_DH_1Hz = resample_mask(GT_mask_DH_25Hz, FPS_mask, frequency_GMAC)

# Ensure GT Mask @ 1 hZ have same size as the Brond AC values 
GT_mask_NDH_1Hz, GMAC_ndh = remove_extra_elements(GT_mask_NDH_1Hz, GMAC_ndh)
GT_mask_DH_1Hz, GMAC_dh = remove_extra_elements(GT_mask_DH_1Hz, GMAC_dh)

In [ ]:
# Plots to ensure the downsampling was correctly done
plot_resampled_arrays(GT_mask_NDH_25Hz, FPS_mask, GT_mask_NDH_1Hz, frequency_GMAC)
plot_resampled_arrays(GT_mask_DH_25Hz, FPS_mask, GT_mask_DH_1Hz, frequency_GMAC)

In [ ]:
# Update the participant dataset file with new arrays of usefull data  
# Add Brond AC
add_attributes_to_participant(participant_data, GMAC_NDH = GMAC_ndh, GMAC_DH = GMAC_dh)
# Add mask @ 1 Hz, @ 2 Hz, @ 25 Hz and  @ 50 Hz
add_attributes_to_participant(participant_data, GT_mask_NDH_1Hz = GT_mask_NDH_1Hz, GT_mask_DH_1Hz = GT_mask_DH_1Hz, 
                             GT_mask_NDH_25Hz = GT_mask_NDH_25Hz, GT_mask_DH_25Hz = GT_mask_DH_25Hz)

In [ ]:
# Plot to visualize GMAC and GT superposed
plot_superposition(GMAC_ndh, 200*GT_mask_NDH_1Hz)
plot_superposition(GMAC_dh, 200*GT_mask_DH_1Hz)

## Individual Optimization

In [ ]:
# Perform k-fold cross-validation for the left wrist data
# Return average optimal AC threshold and average evaluation metrics on this optimal parameter when tested against GT
eval_metrics_opt_ndh, opt_threshold_ndh = k_fold_cross_validation(GMAC_ndh, GT_mask_NDH_1Hz, optimal=True)
# Same but for conventional fixed threshold 
eval_metrics_conv_ndh, conv_threshold_ndh = k_fold_cross_validation(GMAC_ndh, GT_mask_NDH_1Hz, optimal=False)

In [ ]:
# Perform k-fold cross-validation for the right wrist data
# Return average optimal AC threshold and average evaluation metrics on this optimal parameter when tested against GT
eval_metrics_opt_dh, opt_threshold_dh = k_fold_cross_validation(GMAC_dh, GT_mask_DH_1Hz, optimal=True)
# Same but for conventional fixed threshold
eval_metrics_conv_dh, conv_threshold_dh = k_fold_cross_validation(GMAC_dh, GT_mask_DH_1Hz, optimal=False)

In [ ]:
# Optimal AC thresholds for this participant 
print(opt_threshold_ndh, opt_threshold_dh)

In [ ]:
# Create a structure to save all the evaluation metrics 
metrics_dictionary_GMAC = create_metrics_dictionary(eval_metrics_conv_ndh, eval_metrics_conv_dh,
                                               eval_metrics_opt_ndh, eval_metrics_opt_dh)

In [ ]:
metrics_dictionary_GMAC

In [ ]:
# Add metrics for AC threshold evaluation to the JSON file 
add_attributes_to_participant(participant_data, metrics_dictionary_GMAC = metrics_dictionary_GMAC) 
# Add optimal AC parameter to the JSON file 
add_attributes_to_participant(participant_data, optimal_GMAC_NDH = opt_threshold_ndh, optimal_GMAC_DH = opt_threshold_dh) 